In [1]:
import sympy

In [2]:
q = sympy.Symbol('q')
d = sympy.Symbol('d')
L = sympy.Symbol('L')
P = sympy.Symbol('P')
h = sympy.Symbol('h')
b = sympy.Symbol('b')
x = sympy.Symbol('x')
E = sympy.Symbol('E')

In [3]:
subs = {}
#subs[k]=1000
subs[P]=1
subs[x]=.5
subs[b]=.15
subs[h]=.004

In [4]:
# LINKS A and B
#subs[L]=.15
# LINKS C and D
#subs[L]=.165
# AB ANGLE LIMITER
subs[L]=.3

In [5]:
# FROM HARDWARE TESTING
subs[E]=844.45e6

# Approximating a cantilever with a single revolute joint

Cross sectional moment of inertia for a rectangle

In [6]:
I = b*h**3/12

In [7]:
d1 = P*L**3/3/E/I
d1.subs(subs)

0.0133222807744686

In [8]:
q1 = P*L**2/2/E/I

In [9]:
q1.subs(subs)

0.0666114038723429

# 2-Link Approximation: Matching Displacement

In [10]:
k1 = P*L*(1-x)/(sympy.asin(P*L**2/(3*E*I*(1-x))))
k1.subs(subs)

1.68667463715925

The displacement matches

In [11]:
d2 = L*(1-x)*sympy.sin(P*L*(1-x)/k1)
d2.subs(subs)

0.0133222807744686

But the orientation does not

In [12]:
q2 = P*L*(1-x)/k1

In [13]:
q2.subs(subs)

0.0889323860662511

# Matching Theta

In [14]:
k2 = 2*E*I*(1-x)/(L)

Now orientation matches

In [15]:
q3 = P*L*(1-x)/k2

In [16]:
q3.subs(subs)

0.0666114038723429

But displacement does not

In [17]:
d3 = L*(1-x)*sympy.sin(P*L*(1-x)/k2)

In [18]:
d1.subs(subs)

0.0133222807744686

In [19]:
d3.subs(subs)

0.00998432321821591

# Matching Both

In [20]:
del subs[x]

In [21]:
error = []
error.append(d1-d2)
error.append(q1-q2)
error= sympy.Matrix(error)
error = error.subs(subs)
error

Matrix([
[                                                   0],
[0.0666114038723429 - asin(0.044407602581562/(1 - x))]])

In [22]:
import scipy.optimize

In [23]:
f = sympy.lambdify((x),error)

In [24]:
def f2(args):
    a = f(*args)
    b = (a**2).sum()
    return b

In [25]:
sol = scipy.optimize.minimize(f2,[.25])
sol

      fun: 2.247344038984892e-09
 hess_inv: array([[64.87248289]])
      jac: array([9.49405297e-06])
  message: 'Optimization terminated successfully.'
     nfev: 12
      nit: 3
     njev: 6
   status: 0
  success: True
        x: array([0.33331384])

Now add x back to the list of substitutions

In [26]:
subs[x]=sol.x[0]

So a virtual joint at x correctly approximates displacement and orientation.

In [27]:
d2.subs(subs)

0.0133222807744686

In [28]:
q2.subs(subs)

0.0666588100326913

# Results

In [29]:
x.subs(subs)

0.333313835278206

In [30]:
k1.subs(subs)

3.00044134178888

In [31]:
k2.subs(subs)

3.00257670292970

AB: L1 = 0.5 
    
    L2 = 0.1
    
    K = 6

CD: L1 = 0.55 
    
    L2 = 0.11
    
    K = 5.5
    
Angle Limiter (x=0.5): k = 3